# Summary

Try out openai api and test out a few utility functions for the project.

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from enum import IntEnum, Enum
from functools import total_ordering
import matplotlib.pyplot as plt
from multipledispatch import dispatch
import numpy as np
import openai
import os
import pandas as pd
from pathlib import Path

from htools import *
from jabberwocky.config import C
from jabberwocky.utils import load_api_key, load_prompt, openai_auth, \
    bold, print_response

In [22]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [41]:
openai_auth()

In [43]:
C.engines[0]

'ada'

In [26]:
C.prices['curie'], C.prices[2]

(0.006, 0.006)

In [8]:
prompt = load_prompt('how_to')
print(prompt)

How to change a lightbulb:
1. Turn off the light switch. 
2. Using a cloth, push the bulb in, furn anti clockwise and remove it.
3. Insert the new bulb and twist to find the right fit. Push it in and turn clockwise.
4. Turn on the light to test it.

How to tie a tie:
1. Place tie around your neck.
2. Cross the wide end over the thinner end.
3. Run wide end under tie and pull it across again.
4. Pull the wide end through the center.
5. Loop through the knot.
6. Tighten the knot.

How to fry an egg:
1. Crack the Eggs. 
2. Add the Eggs to the Pan. 
3. Cover When the Edges Turn White.
4. Wait until it's done.
5. Serve.

How to


In [153]:
def query_gpt3(prompt, engine_i=0, temperature=0.7, max_tokens=50, 
               logprobs=3, mock=False, **kwargs):
    if mock:
        return '<MOCK>' + ' '.join(prompt.split()[-max_tokens:]) + '</MOCK>'
    res = openai.Completion.create(
        engine=C.engines[engine_i],
        prompt=prompt,
        temperature=temperature,
        max_tokens=max_tokens,
        logprobs=logprobs,
        **kwargs
    )
    # TODO: figure out what to return (what to do about logprobs?)
    return res, res.choices[0].text

In [154]:
# res, text = 
query_gpt3(load_prompt('short_dates'), mock=True)

'<MOCK>Input: 3/1/20 Output: March 1, 2020 Input: 09-04-99 Output: September 4, 1999 Input: 11/01/2017 Output: November 1, 2017 Input: 04/11/21 Output:</MOCK>'

In [126]:
print(text)

 November 11, 2017

Input: 09/25/22
Output: September 25, 2017

Input: 05/07/23
Output: May 7, 2018

Input: 12/18/29
Output: February 18


In [145]:
list(zip(res.choices[0].logprobs.tokens, res.choices[0].logprobs.token_logprobs))

[(' November', -0.91182953),
 (' 11', -0.13010749),
 (',', -0.011717963),
 (' 2017', -0.7044595),
 ('\n', -0.09376717),
 ('\n', -0.07376648),
 ('Input', -0.2312145),
 (':', -0.00809512),
 (' 09', -2.6386774),
 ('/', -0.16320924),
 ('25', -3.823998),
 ('/', -0.010530671),
 ('22', -3.178068),
 ('\n', -0.022007989),
 ('Output', -0.053815383),
 (':', -0.0014988222),
 (' September', -0.7938273),
 (' 25', -0.04676999),
 (',', -0.0053353934),
 (' 2017', -0.9754968),
 ('\n', -0.064693555),
 ('\n', -0.060031462),
 ('Input', -0.20693709),
 (':', -0.0063874256),
 (' 05', -2.7446625),
 ('/', -0.011502205),
 ('07', -3.5464058),
 ('/', -0.0055869846),
 ('23', -0.4102455),
 ('\n', -0.010846776),
 ('Output', -0.039312214),
 (':', -0.0011452552),
 (' May', -0.34696928),
 (' 7', -0.09478703),
 (',', -0.0046168054),
 (' 2018', -0.47485164),
 ('\n', -0.06269102),
 ('\n', -0.05785082),
 ('Input', -0.28975904),
 (':', -0.0064514945),
 (' 12', -2.6183612),
 ('/', -0.0096223075),
 ('18', -3.4610615),
 ('/', -

In [131]:
res.choices[0].logprobs.token_logprobs

[-0.91182953,
 -0.13010749,
 -0.011717963,
 -0.7044595,
 -0.09376717,
 -0.07376648,
 -0.2312145,
 -0.00809512,
 -2.6386774,
 -0.16320924,
 -3.823998,
 -0.010530671,
 -3.178068,
 -0.022007989,
 -0.053815383,
 -0.0014988222,
 -0.7938273,
 -0.04676999,
 -0.0053353934,
 -0.9754968,
 -0.064693555,
 -0.060031462,
 -0.20693709,
 -0.0063874256,
 -2.7446625,
 -0.011502205,
 -3.5464058,
 -0.0055869846,
 -0.4102455,
 -0.010846776,
 -0.039312214,
 -0.0011452552,
 -0.34696928,
 -0.09478703,
 -0.0046168054,
 -0.47485164,
 -0.06269102,
 -0.05785082,
 -0.28975904,
 -0.0064514945,
 -2.6183612,
 -0.0096223075,
 -3.4610615,
 -0.003823817,
 -4.4417586,
 -0.015871562,
 -0.120677136,
 -0.0012529118,
 -4.70235,
 -0.11773417]

In [163]:
res = openai.Completion.create(engine=C.engines[0],
                               prompt=prompt, temperature=.4,
                               max_tokens=100)

In [196]:
text = res['choices'][0]['text']
print(text)

 make a cake:
1. Cut the Cake.
2. Roll it out.
3. Cut out the Design.
4. Cut out the Cake.
5. Roll it out again.
6. Cut out the design.
7. Roll it out again.

How to make a cake:
1. Cut the Cake.
2. Roll it out.
3. Cut out the Design.
4. Roll it out again.
5. Cut


In [197]:
res = openai.Completion.create(engine=C.engines[-1],
                               prompt=prompt, temperature=.4,
                               max_tokens=100)

In [198]:
text = res['choices'][0]['text']
print(text)

 make a cake:
1. Preheat the Oven.
2. Add the Ingredients in the Right Order.
3. Bake in the Oven.
4. Wait until it's done.
5. Serve.

How to make a pizza:
1. Preheat the Oven.
2. Add the Ingredients in the Right Order.
3. Bake in the Oven.
4. Wait until it's done.
5. Serve.




In [202]:
res_dates = openai.Completion.create(
    engine=C.engines[0],
    prompt=load_prompt('short_dates'),
    temperature=.4,
    max_tokens=30,
    logprobs=10
)

In [206]:
print(load_prompt('short_dates'))
print(res_dates['choices'][0]['text'])

Input: 3/1/20
Output: March 1, 2020

Input: 09-04-99
Output: September 4, 1999

Input: 11/01/2017
Output: November 1, 2017

Input: 04/11/21
Output:
 November 11, 2017

Input: 04/11/21
Output: November 11, 2017

Input: 04/11/21



In [12]:
# Moved to test paperspace but don't want to save API credits.
prompt = load_prompt('short_dates')
res = """Input: 04/11/21
Output: November 11, 2017

Input: 04/11/21"""
print_response(prompt, res)

Input: 3/1/20
Output: March 1, 2020

Input: 09-04-99
Output: September 4, 1999

Input: 11/01/2017
Output: November 1, 2017

Input: 04/11/21
Output:Input: 04/11/21
Output: November 11, 2017

Input: 04/11/21


In [45]:
sensitive_txt = 'I\'m going to kill myself.'
sensitive_resp = openai.Completion.create(
    engine='content-filter-alpha-c4', 
    prompt='<|end-of-text|>' + sensitive_txt + '\n--\nLabel:',
    temperature=0,
    max_tokens=1,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    logprobs=10
)

In [116]:
def query_content_filter(text):
    """Wrapper to determine if a piece of text is safe, sensitive, or unsafe.
    Details on these categories are here:
    https://beta.openai.com/docs/engines/content-filter
    This endpoint is free.
    
    Returns
    -------
    tuple[int, float]: First value is predicted class, where 0 is safe,
    1 is sensitive (politics/religion/race/suicide etc.), and 2 is unsafe 
    (profane/prejudiced/otherwise NSFW).
    """
    res = openai.Completion.create(
        engine='content-filter-alpha-c4', 
        prompt='<|end-of-text|>' + text + '\n--\nLabel:',
        temperature=0,
        max_tokens=1,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        logprobs=10
    )
    label = res.choices[0].text
    cls2logp = {x: res.choices[0].logprobs.top_logprobs[0]
                      .get(x, float('-inf'))
                for x in ['0', '1', '2']}
    logp = cls2logp.pop(label)
    # If model is not confident in prediction of 2, choose the next most 
    # likely class. See https://beta.openai.com/docs/engines/content-filter.
    if label == '2' and logp < -.355 and cls2logp:
        label, logp = max(cls2logp.items(), key=lambda x: x[-1])
    return int(label), np.exp(logp)

In [115]:
query_content_filter(sensitive_txt)

(1, 0.6920572815388716)

## Scratch

Experimented with more complex solutions below but these were ridiculously over-complicated for my intended use case. Just trying stuff out for fun.

In [125]:
@total_ordering
@auto_repr
class Engine:
    
    def __init__(self, name, index, price):
        self.name = name
        self.index = index
        self.price = price
        
    def __eq__(self, other):
        return self.index == other.index
    
    def __lt__(self, other):
        return self.index < other.index
    
    def __str__(self):
        return self.name

In [126]:
class Engines:
    
#     i2eng = ['ada', 'babbage', 'curie', 'davinci']
#     eng2price = {'ada': .0008,
#                  'babbage': .0012,
#                  'curie': .006,
#                  'davinci': .06}
    
    i2eng = ['ada', 'babbage', 'curie', 'davinci']
    engines = [Engine(name, i, price) for i, (name, price) in 
               enumerate(zip(i2eng, [.0008, .0012, .006, .06]))]
    
    @dispatch(int)
    def __getitem__(self, i):
        return self.i2eng[i]
    
    @dispatch((list, tuple))
    def __getitem__(self, idx):
        return [self[i] for i in idx]
    
#     @dispatch(int)
#     def price(self, i):
#         return self.eng2price[self[i]]
    
#     @dispatch(str)
#     def price(self, name):
#         return self.eng2price[name]
    
#     def prices(self):
#         return self.eng2price

In [127]:
ENGINES = Engines()

In [128]:
ENGINES.engines

[Engine(name='ada', index=0, price=0.0008),
 Engine(name='babbage', index=1, price=0.0012),
 Engine(name='curie', index=2, price=0.006),
 Engine(name='davinci', index=3, price=0.06)]

In [129]:
ENGINES[0] < ENGINES[2]

True

In [130]:
ENGINES[[0, 2]]

['ada', 'curie']

In [131]:
ENGINES[0, 2]

['ada', 'curie']

In [132]:
for e in ENGINES:
    print(e)

ada
babbage
curie
davinci
